In [ ]:
!pip install openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.2 MB/s eta 0:00:00


In [ ]:
import openai
import json
from datetime import datetime, timedelta
import random
import pandas as pd
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
openai.api_key = 'api_key'

In [ ]:
# Modified seed data
seed_data = [
    {
        'CreatedOn': '2010-07-14',
        'Notification': 30470212,
        'OrderNum': 70091207,
        'ShortText': 'HEAT FROM PUMP MOTOR 12B',
        'FLOC': 'S2CW -2CWE4-MTRD',
        'LongText': (
            "* **************Incident Description**************\n"
            "* Notification 000030470212 Details\n"
            "* ***********************************************************\n"
            "* SUMMARY: VERIFY CURRENT REQUIREMENTS CONTAINED IN\n"
            "* ****PS-AA-120 \"ISSUE IDENTIFICATION AND SCREENING PROCESS\"*****\n"
            "*\n"
            "* 1) Condition Description/Location (The inappropriate action and its\n"
            "negative effect/Include a detailed location description):\n"
            "* At 2051 on 7/14/10 the operator reported heat emanating from\n"
            "the 12B Pump motor. 12B Pump was emergency shut down from\n"
            "Unit 2 Control Room and the heat dissipated. Fire Department and Operations\n"
            "inspection did not reveal any fire but there was a strong odor\n"
            "of burnt insulation. Pump motor temperatures\n"
            "were similar to others currently in operation and no external\n"
            "signs of damage were noted. S2.OP-AB.CW-0001 Pump Abnormality was\n"
            "initiated due to the emergency shutdown of 12B Pump.\n"
            "*\n"
            "* 2) ACTIVITIES, PROCESSES, PROCEDURES INVOLVED:\n"
            "* S2.OP.AB-CW-0001, S2.OP-SO.CW-0001\n"
            "*\n"
            "* 3) WHY DID CONDITION HAPPEN:\n"
            "* Unknown, although pump motors have overheated in the past\n"
            "*\n"
            "* 4) CONSEQUENCES:\n"
            "* No immediate consequences, but potential motor damage could\n"
            "result in loss of efficiency when the unit is back online.\n"
            "*\n"
            "* 5) REQUIREMENTS IMPACTED:\n"
            "* Unavailable pump\n"
            "*\n"
            "* 6) ADVERSE PHYSICAL CONDITION:\n"
            "* Heat from 12B Pump motor, possible internal damage\n"
            "*\n"
            "* 7) WHO WAS NOTIFIED:\n"
            "* Control Room, Fire Department\n"
            "*\n"
            "* 8) KNOWLEDGEABLE INDIVIDUALS:\n"
            "* Johnson, Smith, Lee\n"
            "*\n"
            "* 9) REPEAT OR SIMILAR CONDITION:\n"
            "* Unknown\n"
            "*\n"
            "* 10) IMMEDIATE AND RECOMMENDED ACTIONS:\n"
            "* Maintenance to investigate 12B Pump Motor\n"
            "*\n"
            "* Updated: MICHAEL DOE 07.14.2010 22:45:19\n"
            "* ------------------------------------------------------------\n"
            "* 08/05/2010 08:43:51 JANE DOE (NUSER)\n"
            "* The investigation revealed this was not an electrical failure of the\n"
            "motor. Debris had built up and caused friction, producing the\n"
            "heat. The motor was dismantled, cleaned, and reassembled. The motor\n"
            "is now back in operation.\n"
        )
    },
    {
        'CreatedOn': '2011-08-02',
        'Notification': 30520540,
        'OrderNum': 70098223,
        'ShortText': '15A CW abnormal pump noise',
        'FLOC': 'S2CW -2CWE6',
        'LongText': (
            "* **************Incident Description**************\n"
            "* Notification 000030520540 Details\n"
            "* ***********************************************************\n"
            "* 08/02/2011 09:49:56 INCIDENT REPORT (INCIDENT REPORT)\n"
            "* Report submitted by: Alex Brown 123-456-7890\n"
            "  ENTITY\INCIDENT:\n"
            "* CONDITION DESCRIPTION/LOCATION (THE INAPPROPRIATE ACTION AND ITS\n"
            "NEGATIVE EFFECT/INCLUDE A DETAILED LOCATION DESCRIPTION):\n"
            "* During routine check, noted 15A Pump had unusual noise. Checked inlet\n"
            "pressure and was 12 psig. Checked current - 250. Tide level 80\n"
            "ft. Temperature difference across condenser was similar to other pumps.\n"
            "Based on history, this pump does not typically make noise at 12 psig.\n"
            "*\n"
            "* ACTIVITIES, PROCESSES, PROCEDURES INVOLVED:\n"
            "* Routine check\n"
            "* WHY DID CONDITION HAPPEN?\n"
            "* Unknown, possibly due to debris or pump damage.\n"
            "*\n"
            "* CONSEQUENCES:\n"
            "* Potential reduced efficiency, pump damage\n"
            "*\n"
            "* REQUIREMENTS IMPACTED:\n"
            "* Normal operation\n"
            "*\n"
            "* ADVERSE PHYSICAL CONDITIONS:\n"
            "* High noise level at pump with no known cause\n"
            "*\n"
            "* WHO WAS NOTIFIED:\n"
            "* Supervisor/ INCIDENT TEAM\n"
            "*\n"
            "* KNOWLEDGEABLE INDIVIDUALS:\n"
            "* Incident Team\n"
            "*\n"
            "* REPEAT OR SIMILAR CONDITION:\n"
            "* Yes, multiple\n"
            "*\n"
            "* IMMEDIATE ACTIONS AND RECOMMENDED ACTIONS:\n"
            "* Clean inlet, monitor. If noise persists, further inspection may be required.\n"
            "* 08/02/2011 12:53:52 ALEX BROWN (INCIDENT)\n"
            "* Recommending further inspection to determine cause of noise.\n"
        )
    },
    {
        'CreatedOn': '2015-06-20',
        'Notification': 30694282,
        'OrderNum': 80177886,
        'ShortText': '22A pump higher current post maintenance',
        'FLOC': 'S3CW -3CWE3',
        'LongText': (
            "* **************Incident Description**************\n"
            "* Notification 000030694282 Details\n"
            "* ***********************************************************\n"
            "* 06/20/2015 12:29:00 INCIDENT REPORT (INCIDENT REPORT)\n"
            "* Report submitted by: Charlie Davis ENTITY\INCIDENT:\n"
            "*\n"
            "* CONDITION DESCRIPTION/LOCATION (THE INAPPROPRIATE ACTION AND ITS\n"
            "NEGATIVE EFFECT/INCLUDE A DETAILED LOCATION DESCRIPTION):\n"
            "* In 11/2013, notification 30631539 identified that 21A Pump returned\n"
            "from maintenance with higher current. The issue was resolved under work order 80161318 and\n"
            "procedure revision was issued in Dec 2013 for S3.OP-SO.CW-0001 to\n"
            "update the guidance for higher current. Limits were raised based on data from work order 80167802\n"
            "which was used to replace the 21A pump motor. Since then, 22A pump was\n"
            "serviced in May 2014 under work order 80198850 and returned to service with higher current.\n"
            "It appears that updates to the pump components may be changing the\n"
            "normal current values exhibited by the motors. There is a need to clearly\n"
            "identify the changes made to the pumps and motors so that proper guidance can be developed and\n"
            "procedures updated.\n"
            "*\n"
            "* ACTIVITIES, PROCESSES, PROCEDURES INVOLVED:\n"
            "* Observation of motor current differences and existing procedure guidance in S3.OP-SO.CW-0001.\n"
            "*\n"
            "* WHY DID CONDITION HAPPEN?\n"
            "* Unknown why procedure changes were not identified prior to field changes.\n"
            "*\n"
            "* CONSEQUENCES:\n"
            "* Inadequate guidance in the CW procedures for current limits.\n"
            "*\n"
            "* REQUIREMENTS IMPACTED:\n"
            "* Equipment reliability by ensuring proper current margins are maintained for\n"
            "motors in operation.\n"
            "*\n"
            "* ADVERSE PHYSICAL CONDITIONS:\n"
            "* No\n"
            "*\n"
            "* WHO WAS NOTIFIED:\n"
            "* Supervisor, Maintenance Team\n"
            "*\n"
            "* KNOWLEDGEABLE INDIVIDUALS:\n"
            "* Supervisor, Maintenance Team, Engineers\n"
            "*\n"
            "* REPEAT OR SIMILAR CONDITION:\n"
            "* Yes, notification 30631539 which was resolved but similar event reoccurred.\n"
            "*\n"
            "* IMMEDIATE ACTIONS AND RECOMMENDED ACTIONS:\n"
            "* Engineering to evaluate changes to installed pumps and motors and provide the operations\n"
            "team with technical data to update procedures. Specifically\n"
            "S3.OP-SO.CW-0001\n"
            "* 06/23/2015 11:44:17 JANE DOE (INCIDENT)\n"
            "* The evaluation for the component upgrades has been completed under Rev. 1\n"
            "of 80106141. The motors have been reconditioned and do not require further evaluation.\n"
            "The same guidance provided for 21A applies to additional upgraded pumps installed.\n"
            "Recommend updating the procedure to reflect installation of upgraded pumps.\n"
        )
    }
]

In [ ]:
# Function to format a single seed data entry for the prompt
def format_seed_entry(seed_entry):
    formatted_entry = (
        f"CreatedOn: {seed_entry['CreatedOn']}, Notification: {seed_entry['Notification']}, OrderNum: {seed_entry['OrderNum']}, ShortText: {seed_entry['ShortText']}, "
        f"FLOC: {seed_entry['FLOC']}, LongText: {seed_entry['LongText']}"
    )
    return formatted_entry

In [ ]:
# Function to generate a single synthetic example
def generate_synthetic_example():
    formatted_seed_entries = [format_seed_entry(entry) for entry in seed_data]
    seed_data_text = "\n".join(formatted_seed_entries)

    prompt = (
        "Using the following seed data, generate a new synthetic record with a similar structure and variety:\n"
        f"Seed data:\n{seed_data_text}\n"
        "Make sure the new record has different CreatedOn date, Notification number, Order number, and text variations."
        "Also include all the essential headings, like CONDITION DESCRIPTION, ACTIVITIES, PROCESSES, PROCEDURES INVOLVED, WHY DID CONDITION HAPPEN?, REQUIREMENTS IMPACTED, ADVERSE PHYSICAL CONDITIONS, WHO WAS NOTIFIED, KNOWLEDGEABLE INDIVIDUALS, REPEAT OR SIMILAR CONDITION, IMMEDIATE ACTIONS AND RECOMMENDED ACTIONS"
        "\n\nNew synthetic record:"
    )

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert data generator."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500
    )

    synthetic_record = response.choices[0].message.content
    return synthetic_record.strip()

In [ ]:
# Generate and save 500 synthetic examples
synthetic_records = []
for _ in range(500):
    synthetic_record = generate_synthetic_example()
    synthetic_records.append(synthetic_record)

In [ ]:
import re
# Function to parse the synthetic records into a DataFrame
def parse_synthetic_records(records):
    synthetic_data = []
    for record in records:
        record_dict = {}
        lines = re.split(r',\s(?=\w+:)', record)
        for line in lines:
            if ': ' in line:
                key, value = line.split(': ', 1)
                record_dict[key.strip()] = value.strip().replace('\n', ' ')
        synthetic_data.append(record_dict)
    return pd.DataFrame(synthetic_data)

# Parse the synthetic records into a DataFrame
synthetic_df = parse_synthetic_records(synthetic_records)

# Display and save the synthetic DataFrame
print(synthetic_df)
synthetic_df.to_excel("/content/drive/My Drive/Additional Education/NJIT/Classes/DS677-850 Deep Learning/Project/Work Data/synthetic_data.xlsx", index=False)

      CreatedOn Notification  OrderNum                             ShortText  \
0    2019-03-15     30761040  90231415  Abrupt decrease in 18B Pump pressure   
1    2020-11-18     30958076  80230196      Reduced output from 5B Generator   
2    2018-09-15     30751256  80212345    Rapid frequency change in Pump 18A   
3    2019-11-17     30958230  80523058      Unusual noise from 3B Pump motor   
4    2016-09-15     30751904  80225619                      Fuse M87 tripped   
..          ...          ...       ...                                   ...   
495  2018-12-27     40732176  90234712           Overheating in Converter G4   
496  2020-12-12     30991234  80298250        18A Motor Overheating observed   
497  2019-04-30     30987450  80239876       Unusual vibration in 18B Blower   
498  2021-09-15     30985240  80589752          Abnormal vibration in Fan 5A   
499  2019-04-28     30920832  80287345   29B Pump unusual vibration detected   

                 FLOC                  

In [ ]:
synthetic_df

,CreatedOn,Notification,OrderNum,ShortText,FLOC,LongText,LongText:\n\n* **************Incident Description**************\n* Notification 000030960293 Details\n* ***********************************************************\n* 12/15/2017 13:23:47 INCIDENT REPORT (INCIDENT REPORT)\n* Report submitted by,LongText:\n* **************Incident Description**************\n* Notification 000030802793 Details\n* ***********************************************************\n* 09/15/2016 10:13:15 INCIDENT REPORT (INCIDENT REPORT)\n* Report submitted by,LongText:\n* **************Incident Description**************\n* Notification 000030783410 Details\n* ***********************************************************\n* 11/12/2016 08:35:21 INCIDENT REPORT (INCIDENT REPORT)\n* Report submitted by,LongText:\n* **************Incident Description**************\n* Notification 000030851234 Details\n* ***********************************************************\n* 02/10/2022 08:45:30 INCIDENT REPORT (INCIDENT REPORT)\n* Report submitted by,LongText:\n* **************Incident Description**************\n* Notification 0000309842010 Details\n* ***********************************************************\n* 09/15/2020 13:45:10 INCIDENT REPORT (INCIDENT REPORT)\n* Report submitted by,LongText:\n* **************Incident Description**************\n* Notification 000030865217 Details\n* ***********************************************************\n* 12/04/2022 08:30:01 INCIDENT REPORT (INCIDENT REPORT)\n* Report submitted by,LongText:\n\n* **************Incident Description**************\n* Notification 000030965057 Details\n* ***********************************************************\n* 01/15/2020 07:42:53 INCIDENT REPORT (INCIDENT REPORT)\n* Report submitted by,LongText:\n\n* **************Incident Description**************\n* Notification 000030876362 Details\n* ***********************************************************\n* 09/22/2018 11:17:28 INCIDENT REPORT (INCIDENT REPORT)\n* Report submitted by,LongText:\n* **************Incident Description**************\n* Notification 000030852536 Details\n* ***********************************************************\n* 11/17/2018 15:30:00 INCIDENT REPORT (INCIDENT REPORT)\n* Report submitted by,LongText:\n\n* **************Incident Description**************\n* Notification 000030768921 Details\n* ***********************************************************\n\n* 09/10/2018 14:52:00 INCIDENT REPORT (INCIDENT REPORT)\n* Report submitted by
0,2019-03-15,30761040,90231415,Abrupt decrease in 18B Pump pressure,S4CW -4CWE7,* **************Incident Description**********...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-11-18,30958076,80230196,Reduced output from 5B Generator,S5GN -5GNE2-GENR,* **************Incident Description**********...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-09-15,30751256,80212345,Rapid frequency change in Pump 18A,S4CW -4CWE5,* **************Incident Description**********...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-11-17,30958230,80523058,Unusual noise from 3B Pump motor,S5CW -5CWE2,* **************Incident Description**********...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-09-15,30751904,80225619,Fuse M87 tripped,S4EL -4EL1746,* **************Incident Description**********...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2018-12-27,40732176,90234712,Overheating in Converter G4,S4AC -4ACE7-TG4,* **************Incident Description**********...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,2020-12-12,30991234,80298250,18A Motor Overheating observed,S4CW -4CWE2,* **************Incident Description**********...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,2019-04-30,30987450,80239876,Unusual vibration in 18B Blower,S5BL -3BLE6,* **************Incident Description**********...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,2021-09-15,30985240,80589752,Abnormal vibration in Fan 5A,S5AI -5AIE3,* **************Incident Description*******